In [1]:
from utils.molec_class_files import r14, r32, r50, r125, r134a, r143a, r170, r41, r23, r161, r152a, r152, r134, r143, r116
from utils import atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real, generate_lhs
from fffit.fffit.plot import plot_obj_contour
import os

#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

r41_class = r41.R41Constants()
r23_class = r23.R23Constants()
r161_class = r161.R161Constants()
r152a_class = r152a.R152aConstants()
r152_class = r152.R152Constants()
r143_class = r143.R143Constants()
r134_class = r134.R134Constants()
r116_class = r116.R116Constants()

2024-06-24 14:02:21.716012: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 14:02:22.775607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 14:02:22.775716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 14:02:22.920767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 14:02:23.214510: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 14:02:23.215579: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#Get obj from a set of parameters
import copy
at_class = atom_type.AT_Scheme_11()
save_data = True
obj_choice = "ExpVal"
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
problem_setup = opt_atom_types.Problem_Setup(molec_data_dict, all_gp_dict, at_class, obj_choice)

#Best set from Experiment
#Set parameter set of interest (in this case get the best parameter set)
all_molec_dir = problem_setup.make_results_dir(list(molec_data_dict.keys()))
path_best_sets = os.path.join(all_molec_dir, "best_per_run.csv")
assert os.path.exists(path_best_sets), "best_per_run.csv not found in directory"
all_df = pd.read_csv(path_best_sets, header = 0)
first_param_name = problem_setup.at_class.at_names[0] + "_min"
last_param_name = problem_setup.at_class.at_names[-1] + "_min"
best_set = all_df.loc[0, first_param_name:last_param_name].values
best_real = problem_setup.values_pref_to_real(copy.copy(best_set))
 #Unscale data from 0 to 1 to get correct objective values
at_bounds_pref = at_class.at_bounds_nm_kjmol
theta_guess = values_real_to_scaled(best_real.reshape(1,-1), at_bounds_pref)

In [4]:
repeats = 1
seed =1
setup = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, obj_choice)
rank_parameters, n_data = setup.rank_parameters(best_real)

In [ ]:
opt_num_params, rcc, loss_k, loss_k_params = setup.estimate_opt(theta_guess, rank_parameters, n_data)
print("Optimal number of parameters: ", opt_num_params)
print("RCC: ", rcc)
print("Best Param Set: ", loss_k_params[opt_num_params-1])

In [ ]:
import numpy as np
import numdifftools as nd
fun = lambda x: problem_setup.calc_fxn(x)
dfun = nd.Gradient(fun)
gradient = dfun([theta_guess])

In [ ]:
gradient_scl = gradient*theta_guess
print(gradient_scl.shape)

In [ ]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import lstsq

def rank_parameters(Z):
    # Step 1: Calculate the magnitude of each column in Z
    column_magnitudes = norm(Z, axis=0)
    
    # Initialize variables
    ranked_indices = []  # To keep track of ranked parameter indices
    k = 1
    n, m = Z.shape

    while k <= m:
        if k == 1:
            # Step 2: Identify the most estimable parameter
            max_index = np.argmax(column_magnitudes)
            ranked_indices.append(max_index)
        else:
            # Step 3: Build X_k with the k most estimable columns
            X_k = Z[:, ranked_indices]

            # Predict Z using ordinary least-squares
            Z_hat, _, _, _ = lstsq(X_k, Z)

            # Calculate the residual matrix R_k
            R_k = Z - X_k @ Z_hat

            # Step 4: Calculate the magnitude of each column in R_k
            residual_magnitudes = norm(R_k, axis=0)

            # Step 5: Determine the next most estimable parameter
            # Ensure we pick a column that hasn't been ranked yet
            for idx in np.argsort(-residual_magnitudes):
                if idx not in ranked_indices:
                    ranked_indices.append(idx)
                    break
        
        k += 1  # Step 6: Increase k and repeat

    return ranked_indices

# Example usage
Z = gradient_scl
ranked_params = rank_parameters(Z)
print("Ranked parameters (indices):", ranked_params)
value = [at_class.at_names[param] for param in ranked_params]
print(value)